# Multiple Testing

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt

# StatsModels
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests as mult_test

# SciPy
from scipy.stats import (ttest_1samp, ttest_rel, ttest_ind, t as t_dbn)

# ISLP
from ISLP import load_data